## toBigs 팀 노트북 참고

`psutil` -> 우리는 사용하지 않을 것이므로 해당 셀 무시
- `pip install psutil` 로 설치
- 컴퓨터 리소스 확인 및 정보검색용 크로스 플랫폼 라이브러리
- 크로스 플랫폼?
    - Java: JVM이라는 가상 공간에서 컴파일 되므로 각 os에 맞는 컴파일러로 컴파일할 필요가 없음
    - 게임, 리액트네이티브, 일렉트론 등이 크로스플랫폼 지원 도구다.

### 기상청 데이터 전처리

In [1]:
# load library

# 필수
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# 선택
from tqdm import tqdm   # 진행상황 표시 바, 아랍어로 progress(taqadum)
from glob import glob   # 데이터 로드 (폴더간 이동)
from datetime import datetime, timedelta    # 아마도 시간측정에 사용

# 기본
import math
import time
import random

In [2]:
# 시각화를 위한 설정
# plt.style.use("fivethirtyeight")  # (선택) css로 plt 시각화 결과 제어 / 선택지 있음 (plt)
plt.rc('font', family='NanumBarunGothic')   # (기본) 한글 깨지지 않도록 폰트설ㅈ어
plt.rc('axes', unicode_minus=False)     # (기본) 마이너스 부호 깨지지 않도록
warnings.filterwarnings(action='ignore')    # 필요없는 경고메시지 무시

In [4]:
# 경로 목록
# baseline 참고: https://dacon.io/competitions/official/235720/codeshare/2488?page=1&dtype=recent

root_path = "./data/"
files = sorted(glob(root_path+'*.csv'))

In [5]:
files

['./data/dangjin_fcst_data.csv',
 './data/dangjin_obs_data.csv',
 './data/energy.csv',
 './data/sample_submission.csv',
 './data/site_info.csv',
 './data/ulsan_fcst_data.csv',
 './data/ulsan_obs_data.csv']

In [8]:
dangjin_fcst = pd.read_csv(files[0])
dangjin_obs = pd.read_csv(files[1])
energy = pd.read_csv(files[2])
sample_submission = pd.read_csv(files[3])
site_info = pd.read_csv(files[4])
ulsan_fcst = pd.read_csv(files[5])
ulsan_obs = pd.read_csv(files[6])

In [11]:
dangjin_fcst.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162208 entries, 0 to 162207
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Forecast time  162208 non-null  object 
 1   forecast       162208 non-null  float64
 2   Temperature    162208 non-null  float64
 3   Humidity       162208 non-null  float64
 4   WindSpeed      162208 non-null  float64
 5   WindDirection  162208 non-null  float64
 6   Cloud          162208 non-null  float64
dtypes: float64(6), object(1)
memory usage: 8.7+ MB


In [12]:
dangjin_fcst.head()

,Forecast time,forecast,Temperature,Humidity,WindSpeed,WindDirection,Cloud
0,2018-03-01 11:00:00,4.0,0.0,60.0,7.3,309.0,2.0
1,2018-03-01 11:00:00,7.0,-2.0,60.0,7.1,314.0,1.0
2,2018-03-01 11:00:00,10.0,-2.0,60.0,6.7,323.0,1.0
3,2018-03-01 11:00:00,13.0,-2.0,55.0,6.7,336.0,1.0
4,2018-03-01 11:00:00,16.0,-4.0,55.0,5.5,339.0,1.0
